<a href="https://colab.research.google.com/github/nasibehmohammadi/Thesis/blob/main/1lstm_d3(UCI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

file_path = "/content/d3.csv"
df = pd.read_csv(file_path)

print("🔍 بررسی ۵ سطر اول دیتاست:")
print(df.head())

feature_columns = ["B","C"]
target_column = "A"

X_data = df[feature_columns].values
y_data = df[target_column].values

num_features = X_data.shape[1]

time_steps = 10

X_sequences = []
y_sequences = []

for i in range(len(X_data) - time_steps):
    X_sequences.append(X_data[i : i + time_steps])
    y_sequences.append(y_data[i + time_steps])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = X_train.reshape(-1, num_features)
X_test = X_test.reshape(-1, num_features)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(-1, time_steps, num_features)
X_test = X_test.reshape(-1, time_steps, num_features)

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=2 , batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ دقت مدل روی داده‌های تست: {accuracy * 100:.2f}%")

test_sample = X_test[0].reshape(1, time_steps, num_features)
prediction = model.predict(test_sample)

if prediction > 0.5:
    print("🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.")
else:
    print("✅ ورود ایمن است.")

🔍 بررسی ۵ سطر اول دیتاست:
   A      B      C
0  0  11044  45813
1  1  11044  65766
2  1   9521  72531
3  1   9521  72531
4  1   9521  72531


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
17902/17902 ━━━━━━━━━━━━━━━━━━━━ 383s 21ms/step - accuracy: 0.9841 - loss: 0.0848 - val_accuracy: 0.9851 - val_loss: 0.0781
Epoch 2/2
17902/17902 ━━━━━━━━━━━━━━━━━━━━ 396s 22ms/step - accuracy: 0.9845 - loss: 0.0808 - val_accuracy: 0.9851 - val_loss: 0.0772
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 32s 7ms/step - accuracy: 0.9851 - loss: 0.0773
✅ دقت مدل روی داده‌های تست: 98.51%
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 533ms/step
🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.


In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from sklearn.metrics import f1_score

# 1️⃣ پیش‌بینی بر روی مجموعه تست
y_pred = model.predict(X_test)

# 2️⃣ تبدیل احتمالات به برچسب‌های 0 و 1 (اگر مقدار احتمال > 0.5 باشد، کلاس 1 است، در غیر این صورت 0)
y_pred_classes = (y_pred > 0.5).astype(int)

# 3️⃣ محاسبه F1-Score
f1 = f1_score(y_test, y_pred_classes)

print(f"🎯 F1-Score مدل: {f1:.4f}")

4476/4476 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step
🎯 F1-Score مدل: 0.9925
